In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd

import ssl
ssl._create_default_https_context = ssl._create_stdlib_context

In [4]:
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options

# Tạo đối tượng ChromeOptions
chrome_options = Options()

# Cấu hình các tùy chọn
chrome_options.headless = False  # Hiển thị trình duyệt
# chrome_options.add_argument('--start-maximized')
chrome_options.add_argument('disable-infobars')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')

# Sử dụng chrome_options thay vì options
driver = uc.Chrome(chrome_options=chrome_options)

# Mở rộng cửa sổ trình duyệt
driver.maximize_window()

# Truy cập Shopee
driver.get("https://shopee.vn")


In [ ]:
quynhanhnguyenngoc081@gmail.com
Anh123123

In [5]:
import pickle

def saveCookie(driver, cookieName):
    with open('.' + '/' + cookieName, 'wb') as filehandler:
        pickle.dump(driver.get_cookies(), filehandler)
saveCookie(driver, 'cookies.pkl')


In [193]:
import pandas as pd

# Đọc file CSV
df = pd.read_csv('cleaned_file.csv')

# Đếm số giá trị không bị thiếu trong cột 'name_product'
count_non_missing = df['name_product'].notna().sum() - (df['name_product'] == "").sum()

print(f"Số dòng có giá trị ở cột 'name_product': {count_non_missing}")



Số dòng có giá trị ở cột 'name_product': 762


In [191]:

#Tiền xử lý dữ liệu
import pandas as pd
df = pd.read_csv('shopee.csv')
df.rename(columns={'Người theo dõi': 'followers'}, inplace=True)
df.rename(columns={'Sản phẩm': 'product_count'}, inplace=True)
df.rename(columns={'tham gia': 'join_time'}, inplace=True)
df.rename(columns={'tỉ lệ phản hồi': 'respond_rate'}, inplace=True)
df.rename(columns={'thời gian phản hồi': 'respond_time'}, inplace=True)
df.rename(columns={'Đánh giá': 'evaluation'}, inplace=True)

df.drop(columns=['price'], inplace=True)
# Hàm chuyển đổi
def convert_price(value):
    value_str = str(value)  # Đảm bảo giá trị là kiểu chuỗi
    value_str = value_str.split('-')[-1].strip()
    if '₫' in value_str:
        number = value_str.replace('₫', '').replace('.', '')
        return float(number)

def convert_value(value):
    value_str = str(value)  # Đảm bảo giá trị là kiểu chuỗi
    if 'k' in value_str:
        # Xóa 'k' và thay thế ',' bằng '.'
        number = float(value_str.replace('k', '').replace(',', '.'))
        return number * 1000

    elif 'tr' in value_str:
        number = float(value_str.replace('tr', '').replace(',', '.'))
        return number * 1000000

    else:
        # Chuyển đổi chuỗi không có 'k'
        return float(value_str.replace(',', '.'))
#Hàm chuyển đổi quantiy
def convert_quantity(value):
    if pd.isna(value):  # Kiểm tra nếu giá trị là NaN
        return None  # Hoặc bạn có thể trả về một giá trị mặc định khác
    value_str = str(value)  # Đảm bảo giá trị là kiểu chuỗi
    quantity = value_str.split()[0]  # Tách và lấy phần đầu tiên
    return quantity
def convert_like_count(value):
    value_str = str(value)  # Đảm bảo giá trị là kiểu chuỗi
    value_str = value_str.split()[-1]
    value_str = value_str.replace('(', '')
    value_str = value_str.replace(')', '')
    if 'k' in value_str:
        # Xóa 'k' và thay thế ',' bằng '.'
        number = float(value_str.replace('k', '').replace(',', '.'))
        return number * 1000
    elif 'tr' in value_str:
        number = float(value_str.replace('tr', '').replace(',', '.'))
        return number * 1000000

    else:
        # Chuyển đổi chuỗi không có 'k'
        return float(value_str.replace(',', '.'))

df['evaluation'] = df['evaluation'].apply(convert_value)
df['product_count'] = df['product_count'].apply(convert_value)
df['sale_count'] = df['sale_count'].apply(convert_value)
df['review_count'] =df['review_count'].apply(convert_value)
df['quantity'] = df['quantity'].apply(convert_quantity)
df['like_count'] = df['like_count'].apply(convert_like_count)
df['followers'] = df['followers'].apply(convert_value)
df['price_origin'] = df['price_origin'].apply(convert_price)
df['fee_delivery'] = df['fee_delivery'].apply(convert_price)
df.to_csv('cleaned_file.csv', index=False)


In [192]:
import pandas as pd
from pymongo import MongoClient

# Kết nối đến MongoDB
client = MongoClient("mongodb://localhost:27017/")
db_shopee = client["shopee"]  # Thay thế bằng tên database của bạn
collection_shopee = db_shopee["collection_shopee"]  # Thay thế bằng tên collection của bạn

# Đọc file CSV
df = pd.read_csv('cleaned_file.csv')

# Chuyển đổi DataFrame thành danh sách các dictionary
data = df.to_dict(orient="records")

# Chèn dữ liệu vào MongoDB
collection_shopee.insert_many(data)

print("Dữ liệu đã được lưu vào MongoDB.")


Dữ liệu đã được lưu vào MongoDB.
